In [1]:
from flask import Flask, request, render_template
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

app = Flask(__name__)

# Load movies data
movies = pd.read_csv('https://raw.githubusercontent.com/codeheroku/Introduction-to-Machine-Learning/master/Building%20a%20Movie%20Recommendation%20Engine/movie_dataset.csv')

# Select features for recommendation (you can adjust this based on your dataset)
features = ['keywords', 'cast', 'genres', 'director']

# Fill NaN values with empty string
for feature in features:
    movies[feature] = movies[feature].fillna('')

# Function to combine the selected features into a single string
def combine_features(row):
    return row['keywords'] + ' ' + row['cast'] + ' ' + row['genres'] + ' ' + row['director']

# Apply function to create a new 'combined_features' column
movies['combined_features'] = movies.apply(combine_features, axis=1)

# Create a count matrix from the combined features column
cv = CountVectorizer()
count_matrix = cv.fit_transform(movies['combined_features'])

# Compute cosine similarity based on the count matrix
cosine_sim = cosine_similarity(count_matrix)

# Function to get movie recommendations based on title
def get_recommendations(title, cosine_sim):
    # Get index of the movie that matches the title
    idx = movies[movies['title'].str.lower() == title.lower()].index[0]

    # Get pairwise similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort movies based on similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get top 10 recommendations (excluding the movie itself)
    sim_scores = sim_scores[1:11]

    # Get movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return top 10 most similar movies
    return movies['title'].iloc[movie_indices]

@app.route('/')
def home():
    return render_template('home.html')

@app.route('/recommend', methods=['POST'])
def recommend():
    movie = request.form['movie']
    recommendations = get_recommendations(movie, cosine_sim)
    return render_template('home.html', movie=movie, recommendations=recommendations)

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

C:\Users\dell\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
